In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [2]:
train_data, validation_data, test_data = tfds.load(name = 'imdb_reviews',
                                                  split = ('train[:60%]', 'train[60%:]', 'test'),
                                                  as_supervised = True)

In [3]:
train_example_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [4]:
train_example_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [5]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>

# Embedding + Model

In [6]:
class HubWrapper(tf.keras.layers.Layer):
  #Wraps a TensorFlow‑Hub SavedModel so Keras3 sees it as a regular Layer.

  def __init__(self, url: str, trainable: bool = False, **kwargs):
    super().__init__(trainable = trainable, **kwargs)
    self.url = url
    self._hub_model = None #lazy-load in build()

  def build(self, input_shape):
    self._hub_model = hub.load(self.url)
    self._hub_model = tf.function(self._hub_model)
    super().build(input_shape)

  def call(self, inputs):
    outputs = self._hub_model(inputs)

    if isinstance(outputs, dict):
      outputs = outputs.get("default", list(outputs.values())[0])

    return outputs

In [7]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = HubWrapper(embedding, dtype = tf.string, trainable = True, name = "text_embedding")

In [ ]:
    model = tf.keras.Sequential()
    model.add(hub_layer)
    model.add(tf.keras.layers.Dense(16, activation = 'relu'))
    model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_embedding (HubWrapper)     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
model.compile(optimizer = 'adam',
              loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
              metrics = ['accuracy'])

In [11]:
history = model.fit(train_data.shuffle(10000).batch(100),
                    epochs = 25,
                    validation_data = validation_data.batch(100),
                    verbose = 1)

Epoch 1/25



C:\Users\Kshitij Jaywant More\AppData\Roaming\Python\Python312\site-packages\keras\src\backend\tensorflow\nn.py:789: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.5431 - loss: 1.0809 - val_accuracy: 0.6571 - val_loss: 0.6241
Epoch 2/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.6616 - loss: 0.6189 - val_accuracy: 0.6886 - val_loss: 0.5937
Epoch 3/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.6925 - loss: 0.5847 - val_accuracy: 0.7041 - val_loss: 0.5741
Epoch 4/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.7027 - loss: 0.5749 - val_accuracy: 0.7047 - val_loss: 0.5688
Epoch 5/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.7057 - loss: 0.5687 - val_accuracy: 0.7128 - val_loss: 0.5604
Epoch 6/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.7067 - loss: 0.5655 - val_accuracy: 0.7183 - val_loss: 0.5581
Epoch 7/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.7192 - loss: 0.5492 - val_accuracy: 0.7179 - val_loss: 0.5569
Epoch 8/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.7142 - loss: 0.5541 - val_accuracy: 0.718

In [16]:
results = model.evaluate(test_data.batch(100), verbose = 2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

250/250 - 4s - 16ms/step - accuracy: 0.7214 - loss: 0.5518
loss: 0.552
compile_metrics: 0.721
